# Sains Data di Awan: Cara "Azure ML SDK"

## Pengenalan

Dalam buku nota ini, kita akan belajar cara menggunakan Azure ML SDK untuk melatih, menyebarkan, dan menggunakan model melalui Azure ML.

Prasyarat:
1. Anda telah mencipta ruang kerja Azure ML.
2. Anda telah memuat naik [set data Kegagalan Jantung](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) ke dalam Azure ML.
3. Anda telah memuat naik buku nota ini ke dalam Azure ML Studio.

Langkah-langkah seterusnya adalah:

1. Cipta satu Eksperimen dalam Ruang Kerja yang sedia ada.
2. Cipta kluster Komputer.
3. Muatkan set data.
4. Konfigurasikan AutoML menggunakan AutoMLConfig.
5. Jalankan eksperimen AutoML.
6. Terokai hasil dan dapatkan model terbaik.
7. Daftarkan model terbaik.
8. Sebarkan model terbaik.
9. Gunakan titik akhir.

## Import khusus untuk Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Memulakan Ruang Kerja
Mulakan objek ruang kerja daripada konfigurasi yang disimpan. Pastikan fail konfigurasi terdapat di .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Cipta eksperimen Azure ML

Mari kita cipta satu eksperimen bernama 'aml-experiment' dalam ruang kerja yang baru sahaja kita inisialisasikan.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Membuat Kluster Pengiraan
Anda perlu mencipta [sasaran pengiraan](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) untuk menjalankan AutoML anda.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Data
Pastikan anda telah memuat naik set data ke Azure ML dan bahawa kunci mempunyai nama yang sama seperti set data.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Konfigurasi AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Larian AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Melancarkan Model Terbaik

Jalankan kod berikut untuk melancarkan model terbaik. Anda boleh melihat status pelancaran di portal Azure ML. Langkah ini mungkin mengambil masa beberapa minit.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Menggunakan Endpoint
Anda boleh menambah input kepada sampel input berikut.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat penting, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
